# DEMO for dimensions and parameters calculation

Let's build a simple CNN, visualize the model and figure out how the dimesions and parameters are calculated. It's great if you've used Tensorflow Functional API before, else there will be sufficient details to follow this notebook easily. 

For this demo we'll build a very simple classifier with 2 sets of convolution, batch normalization and maxpooling layers followed by a single dense layer. We'll train the model for 5 epochs on MNSIT dataset. We won't concern ourselves with the training results as such. 

To run this notebook, I am using the following package versions:

Tensorflow: 2.8.0

Tensorflow datasets: 4.6.0

There might be minor variations in the code for different versions of these packages.

Note: Tensorboard isn't an essential for this demo but just adding it cause it looks cool!

## Import necessary packages

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import tensorboard

In [2]:
%load_ext tensorboard

In [3]:
def basic_cnn(inputs):
    '''
       Defining a basic CNN model for classification with 2 conv2d, 2 batch norm, 2 max pooling and one dense layer.
       Parameters
       inputs: input images
       Output:
       output_layer: output from final dense layer. Number of classes = 10
    '''
    
    ### First set of conv2d and batchnorm
    conv_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(inputs)
    bn1 =  tf.keras.layers.BatchNormalization()(conv_1)
    
    ### First maxpool
    max_pool_1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(bn1)
    
    ### Second set of conv2d and batchnorm
    conv_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(max_pool_1)
    bn2 =  tf.keras.layers.BatchNormalization()(conv_2)
    
    ### Second max pool
    max_pool_2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv_2)
    
    ### Flatten it for the dense layer
    flatten_layer = tf.keras.layers.Flatten()(max_pool_2)
    
    ### Two dense layers and output
    first_dense = tf.keras.layers.Dense(128, activation='relu')(flatten_layer)
    output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(first_dense)
    return output_layer

In functional API the inputs and outputs hae to be put together using tf.keras.Models. This makes is flexible to handle multiple inputs and outputs.

In [4]:
def cnn_model():
    '''Put the model together
       Output
       model: The entire cnn model'''
    #Define the inputs
    inputs = tf.keras.layers.Input(shape=(28, 28, 1,))
    #Get the encoder output
    cnn_op= basic_cnn(inputs)
    #Store the outputs of the bottleneck layer for visualisation and analysis
    #Put it together and get your model
    model = tf.keras.Model(inputs =inputs, outputs=cnn_op)
    return model
     

In [5]:
# Lets call the model now
basic_cnn_model = cnn_model()
basic_cnn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0     

## Data Preparation

Pretty straightforward here with MNIST. Just normalize and we are set to go. Just fetch the train dataset and run a few epochs.

In [6]:
def preprocess(features):
    # Normalize the images and return (image, label) pairs
    return tf.cast(features['image'], tf.float32) / 255., features['label']

dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
dataset = dataset.map(preprocess).batch(32)

## Compile the model with optimizer, loss and metrics. Note: Custom functions can be used here as well
basic_cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Initialize callbacks to work for Tensorboard (not the main objective here, just showcasing)
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# train the model for 5 epoch(just this once for the purpose of demo!)
history=basic_cnn_model.fit(dataset, epochs=5, callbacks=[tensorboard_callback])
%tensorboard --logdir logs  --host localhost


Epoch 1/5
1875/1875 [==============================] - 55s 29ms/step - loss: 0.1039 - accuracy: 0.9682
Epoch 2/5
1875/1875 [==============================] - 56s 30ms/step - loss: 0.0403 - accuracy: 0.9876
Epoch 3/5
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0269 - accuracy: 0.9917
Epoch 4/5
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0214 - accuracy: 0.9930
Epoch 5/5
1875/1875 [==============================] - 63s 34ms/step - loss: 0.0165 - accuracy: 0.9950


Reusing TensorBoard on port 6006 (pid 37540), started 0:05:48 ago. (Use '!kill 37540' to kill it.)